In [ ]:
# from mnist import train_images, train_labels, test_images, test_labels

n_feature = 28 * 28
n_class = 10

# train_data = train_images().reshape(-1, n_feature).astype('float64') / 255.
# train_labels = train_labels()
# test_data = test_images().reshape(-1, n_feature).astype('float64') / 255.
# test_labels = test_labels()

In [ ]:
# 用tensorflow做逻辑回归
import tensorflow as tf
import numpy as np
import pandas as pd


In [ ]:


from tensorflow.keras.datasets import mnist
(train_data, train_labels), (test_data, test_labels) = mnist.load_data()
# 数据集拆分
train_data, test_data = np.array(train_data, np.float32), np.array(test_data, np.float32)
# Flatten images to 1-D vector of 784 features (28*28).
train_data, test_data = train_data.reshape([-1, n_feature]), test_data.reshape([-1, n_feature])
# Normalize images value from [0, 255] to [0, 1].
train_data, test_data = train_data / 255., test_data / 255.

train_data = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
train_data = train_data.repeat().shuffle(5000).batch(1000).prefetch(1)

In [ ]:
import tensorflow as tf

def softmax(x):
    exp = tf.exp(x)
    return exp / tf.reduce_sum(exp, 1, keepdims=True)

def net(x, w, b):
    return softmax(tf.matmul(x, w) + b)

# Logistic regression (Wx + b).
# def net(x, w, b):
#     # Apply softmax to normalize the logits to a probability distribution.
#     return tf.nn.softmax(tf.matmul(x, w) + b)

# def cross_entropy(y_pred, y_true):
#     # Encode label to a one hot vector.
#     y_true = tf.one_hot(y_true, depth=n_class)
#     # Clip prediction values to avoid log(0) error.
#     y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
#     # Compute cross-entropy.
#     return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred),1))
def cross_entropy(y_hat, y):
    y_hat = tf.clip_by_value(y_hat, 1e-9, 1.)
    return -tf.math.log(tf.boolean_mask(y_hat, tf.one_hot(y, depth=y_hat.shape[-1])))

# Accuracy metric.
# def accuracy(y_pred, y_true):
#     # Predicted class is the index of highest score in prediction vector (i.e. argmax).
#     correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
#     return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
def accuracy(y_hat, y):
    y_hat = tf.argmax(y_hat, axis=1)
    cmp = tf.cast(y_hat, y.dtype) == y
    return tf.reduce_sum(tf.cast(cmp, y.dtype)) / len(y)

eta = 1e-2
w = tf.Variable(tf.random.normal((n_feature, n_class)))
b = tf.Variable(tf.zeros((n_class)))

optimizer = tf.keras.optimizers.SGD(eta)
losses = list()
for i, (batch_x, batch_y) in enumerate(train_data.take(4000)):
    with tf.GradientTape() as g:
        y_hat = net(batch_x, w,  b)
        loss = cross_entropy(y_hat, batch_y)
    grads = g.gradient(loss, [w, b])
    optimizer.apply_gradients(zip(grads, [w, b]))
    ty_hat = net(test_data, w, b)
    acc = accuracy(ty_hat, test_labels)
    losses.append(loss.numpy().mean())
    if i % 200 == 0:
        print(f'loss: {loss.numpy().mean()}, test acc: {acc}')


In [ ]:
print(tf.argmax(net(test_data[:10], w, b)))
print(test_labels[:10])

In [ ]:
from matplotlib import pyplot as plt
plt.plot(losses)